# Jupyter Notebook Testing

Jupyter Notebooks use cells that can easily be re ran and is very convenient for testing things out.
It's important to note that to be able to run these cells correctly, the dependencies need to be properly installed.
This means that the interpreter being used must point to the Python the one that is in your [venv](./.venv) created by poetry for this project.

**Note**: if you make changes to the source code, you will need to press `Restart` above to restart the kernel to ensure the changes occur here.
![image](../pictures/restartJupyterKernel.png)

In [1]:
import logging

logging.basicConfig(level=logging.INFO)
from engine.types.industry_type import IndustryType
from engine.types.demographic import Demographic


policies = {
    "corporate_income_tax": {itype.value: 0.0 for itype in IndustryType},
    "personal_income_tax": 0.0,
    "sales_tax": {itype.value: 0.0 for itype in IndustryType},
    "property_tax": 0.0,
    "tariffs": {itype.value: 0.0 for itype in IndustryType},
    "subsidies": {itype.value: 0.0 for itype in IndustryType},
    "minimum_wage": 0.0,
}

demographics = {
    demo: {
        "income": {"mean": 300 + (i * 500), "sd": 10 + (i * 100)},
        "proportion": 1 / len(Demographic),
        "unemployment_rate": 0.1 - (i * 0.02),
        "spending_behavior": {
            itype.value: 1 / len(IndustryType) for itype in IndustryType
        },
        "current_money": {"mean": 500 + (i * 2000), "sd": 100},
    }
    for i, demo in enumerate(Demographic)
}

In [2]:
from engine.interface.controller import ModelController
controller = ModelController()
model_id = controller.create_model(num_people=100,demographics=demographics, starting_policies=policies,inflation_rate=0.001)

Incomes for lowerclass: [304.81963088 301.98240608 302.61496102 303.74095875 303.33275802
 299.75479647 298.28939663 309.23887583 304.07917532 291.80694979
 313.28921598 312.3017363  299.29845607 318.25307829 308.29501422
 297.50081467 284.38592748 287.95288403 311.8068449  295.46165041
 310.79026779 299.35481081 295.10117933 295.79989017 306.08726809
 296.19687853 311.92576013 288.7979997  299.91007133 301.5156026
 298.56198424 292.93437657 300.98166478]
Incomes for middleclass: [ 711.91239428  887.35449471  693.73322123  753.91443792  888.88955294
 1109.9755459  1008.05693509  929.77950765  726.29604813  707.8472514
  776.28623068  726.20575605  942.28653237  754.96212263  731.17908642
  801.59445781  703.56664627  739.69092865  815.47984336  918.17050111
  698.29841622  723.01266004  764.06194633  732.55920249  822.63957015
 1037.63817787  830.50237832  766.92952642  773.40751184  566.79689589
  537.53533076  924.66216434  529.59337377]
Incomes for upperclass: [1296.32061612 1450.91

In [3]:
controller.step_model(model_id,time=1)

INFO:MESA.mesa.model:calling model.step for timestep 1 
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Producing goods...NOT IMPLEMENTED


In [4]:
indicators = controller.get_indicators(model_id, start_time=1, end_time=0)
print(indicators)

   Week  Unemployment  GDP  IncomePerCapita  MedianIncome  HooverIndex  \
0     1           1.0    0       801.000001    801.000028            0   

   LorenzCurve  
0            0  
